# Objective<br>
<ol><li>Learn about POS tagging with Markov Models.</li>
<li>Implement the model from scratch.</li>
<li>Calculate the best sentence.</li></ol>
** Before running this notebook please read NLP text book chapter 5.**

<img src="ptag.png">

<img src = "ptag2.png"><br>
** Images and content based on Martin Jurafsky  textbook and slides by Prof. Barbara Di Eugenio.** 

In [1]:
from nltk.tag.stanford import StanfordPOSTagger
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
import nltk

In [2]:
# define my train corpus
text = ("I fed my cat. "
	+"My friend was interrogated by the fed. "
	+"Make sure he is a fed. "
	+"Water level raises always when the snow melts more than 3 feet. "
	+"raises is a good vacation destination. "
	+"Computer science is of great interest to me. "
	+"MNP bank raises interest rates again by 3 percent. "
	+"I thought this book might interest Simon. "
	+"Water level rises at the rate of 3 feet every month. "
	+"the ambassador rates a bulletproof car and a police escort "
	+"Jeff still rates as one of the nicest people I have ever met. "
	+"efforts were made to interest her in a purchase. "
	+"the merger is not contrary to the public interest. "
	+"the selfish pursuit of one's own welfare self interest. "
	+"holders of voting rights must disclose their interests. "
	+"Bring me 1 book on global warming. "

	).lower()

In [3]:
# declare my test sentence. In this example we make sure we are using the words in the train corpus.
# This is done to ensure that there are no zero probabilities. We usually train with a larger corpus or use Laplace smoothing.
test = "fed raises interest rates again by 3 percent".split(" ")

In [4]:
# Tokenizing and tagging
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)
print(len(tagged_tokens))
N = len(tagged_tokens)
sentences = sent_tokenize(text)
sentences = [s.replace("."," ") for s in sentences]
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
print(len(tagged_sentences))

151
15


In [5]:
# Initialize start and end positions
pos_dict = {}
word_tags = {}
transtition_tags = {}
pos_dict["start"] = 15
pos_dict["end"] = 15

In [6]:
# Fill the word_tags dict
for tags in tagged_tokens:
	if not tags[1] in pos_dict:
		pos_dict[tags[1]] = 0
	pos_dict[tags[1]]+=1
	if not tags[0]+"|"+tags[1] in word_tags:
		word_tags[tags[0]+"|"+tags[1]] = 0
	word_tags[tags[0]+"|"+tags[1]] += 1

In [7]:
tags = list(pos_dict.keys())
tags.remove("end")
tags.append("end")


In [9]:
# Fill the trasntition probs dictionary
for sentence in tagged_sentences:
	for t in range(len(sentence)):
		if t is 0:
			if not "start|"+sentence[t][1] in transtition_tags:
				transtition_tags["start|"+sentence[t][1]] = 0
			transtition_tags["start|"+sentence[t][1]]+=1
			continue
		elif t is len(sentence)-1:
			
			if not sentence[t][1]+"|end" in transtition_tags:
				transtition_tags[sentence[t][1]+"|end"] = 0
			transtition_tags[sentence[t][1]+"|end"]+=1
			continue

		if not sentence[t-1][1]+"|"+sentence[t][1] in transtition_tags:
			transtition_tags[sentence[t-1][1]+"|"+sentence[t][1]] = 0
		transtition_tags[sentence[t-1][1]+"|"+sentence[t][1]] += 1

In [10]:
# Fill Emission Probs dictionary and write to file
emission_probs = {}
with open("emission_probs.txt", "w") as file:
	for key,value in word_tags.items():
		emission_probs [key] = float(value)/pos_dict[key.split("|")[1]]
		file.write(key+" : "+str(emission_probs[key])+"\n")

In [ ]:
## Fill and write transtition probs into a file
transtition_probs = {}
with open("transtition_probs.txt",'w') as file:
	for key,value in transtition_tags.items():
		transtition_probs[key] = float(value)/pos_dict[key.split("|")[0]]
		file.write(key+" : "+str(transtition_probs[key])+"\n")

# Excercise: <br>
** You should now have the emission and transtion probabilities in the files. According to the formula discussed above calculate all possible taggings and give the best sentence and its probability. **
